In [2]:
from sentence_transformers import SentenceTransformer
import numpy as np
import json
import pandas as pd
from tqdm.notebook import tqdm

/workspaces/dev/venv/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [ ]:
model = SentenceTransformer(
    "Alibaba-NLP/gte-multilingual-base",
    trust_remote_code=True
)

Some weights of the model checkpoint at Alibaba-NLP/gte-multilingual-base were not used when initializing NewModel: {'classifier.weight', 'classifier.bias'}
- This IS expected if you are initializing NewModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing NewModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [23]:
df = pd.read_json("../data/full_faq.json")
df.head()

,question,answer,category
0,Di mana ada lokasi Rumah Sakit Siloam?,Ada 40 Rumah Sakit modern yang terdiri dari 13...,FAQ Website
1,Apa jenis kamar rawat inap yang tersedia?,Siloam Hospitals menawarkan banyak pilihan jen...,FAQ Website
2,Apa standar keamanan suplai darah yang diambil...,Untuk memastikan keamanan dan kualitas suplai ...,FAQ Website
3,Fasilitas apa yang dimiliki pusat keunggulan R...,Kardiologi: Di Cardiac Center kami terdapat Un...,FAQ Website
4,Apa standar kualitas yang dipatuhi oleh Siloam...,Siloam Hospitals mematuhi standar kualitas dar...,FAQ Website


In [27]:
vectors = model.encode(
    [row.question + ". " + row.answer for row in df.itertuples()],
    show_progress_bar=True,
)

Batches: 100%|██████████| 11/11 [00:44<00:00,  4.00s/it]


In [28]:
vectors.shape


(349, 768)

In [29]:
np.save("faq_vectors.npy", vectors, allow_pickle=False)

In [30]:
# Import client library
from qdrant_client import QdrantClient
from qdrant_client.models import VectorParams, Distance

client = QdrantClient("http://localhost:6333")


In [31]:
if not client.collection_exists("faq"):
    client.create_collection(
        collection_name="faq",
        vectors_config=VectorParams(size=768, distance=Distance.COSINE),
    )

In [47]:
fd = open("../data/full_faq.json")

# payload is now an iterator over startup data
payload = map(json.loads, fd)

# Load all vectors into memory, numpy array works as iterable for itself.
# Other option would be to use Mmap, if you don't want to load all data into RAM
vectors = np.load("./faq_vectors.npy")

In [51]:
client.upload_collection(
    collection_name="faq",
    vectors=vectors,
    payload=payload,
    ids=None,  # Vector ids will be assigned automatically
    batch_size=768,  # How many vectors will be uploaded in a single request?
)

JSONDecodeError: Expecting value: line 2 column 1 (char 2)

In [20]:
from qdrant_client import QdrantClient
from sentence_transformers import SentenceTransformer


class NeuralSearcher:
    def __init__(self, collection_name):
        self.collection_name = collection_name
        # Initialize encoder model
        self.model = SentenceTransformer(
            "Alibaba-NLP/gte-multilingual-base",
            trust_remote_code=True
        )
        # initialize Qdrant client
        self.qdrant_client = QdrantClient("http://localhost:6333")
    
    def search(self, text: str):
        # Convert text query into vector
        vector = self.model.encode(text).tolist()

        # Use `vector` for search for closest vectors in the collection
        search_result = self.qdrant_client.query_points(
            collection_name=self.collection_name,
            query=vector,
            query_filter=None,  # If you don't want any filters for now
            limit=5,  # 5 the most closest results is enough
        ).points
        # `search_result` contains found vector ids with similarity scores along with the stored payload
        # In this function you are interested in payload only
        payloads = [hit.payload for hit in search_result]
        return payloads


In [21]:
# Initialize the searcher with the collection name
searcher = NeuralSearcher(collection_name="startups")

# Perform a search with a sample query
results = searcher.search("AI startup in healthcare")

# Print the search results
for result in results:
    print(result)

Some weights of the model checkpoint at Alibaba-NLP/gte-multilingual-base were not used when initializing NewModel: {'classifier.weight', 'classifier.bias'}
- This IS expected if you are initializing NewModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing NewModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


{'name': 'GiveForward', 'images': 'https://d1qb2nb5cznatu.cloudfront.net/startups/i/1374-e472ccec267bef9432a459784455c133-thumb_jpg.jpg?buster=1397666635', 'alt': 'GiveForward -  health care startups crowdfunding', 'description': "Crowdfunding for medical and life events\nGiveForward lets anyone to create a free fundraising page for a friend or loved one's uncovered medical bills, memorial fund, adoptions or any other life events in five minutes or less. Millions of families have used GiveForward to raise more than $165M to let ...", 'link': 'http://giveforward.com', 'city': 'Chicago'}
{'name': 'CancerIQ', 'images': 'https://d1qb2nb5cznatu.cloudfront.net/startups/i/183714-444e9d7ff2951c9713781f56d79f31e7-thumb_jpg.jpg?buster=1378406599', 'alt': 'CancerIQ -  health care information technology big data ventures for good personal health', 'description': 'Predictive analytics to eliminate cancer (Rock Health)\nOur team has modeled proven cancer genetics workflows at top academic centers, a